In [ ]:
!unzip "/content/train-v1.1.json.zip"

Archive:  /content/train-v1.1.json.zip
  inflating: train-v1.1.json         


In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge

In [ ]:
!pip install blue

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
import evaluate

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import json

In [ ]:
with open('/content/train-v1.1.json') as f:
  data = json.load(f)


In [ ]:
data

In [ ]:
data['data'][0]['paragraphs'][0]['qas'][0]['question']

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [ ]:
articles=[]

for article in data['data']:
  for paragraph in article['paragraphs']:
    for qa in paragraph['qas']:
      inputs={"context": paragraph['context'],'question': qa['question'],'answers':qa['answers'][0]['text']}
      articles.append(inputs)


In [ ]:
articles[1]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': 'a copper statue of Christ'}

In [ ]:
data=pd.DataFrame(articles)

In [ ]:
data.head()

,context,question,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary


In [ ]:
t_len=0
for i in data['answers']:
  if len(i.split())>t_len:
    t_len=len(i.split())
print(t_len)

43


In [ ]:
q_len=0
for i in data['question']:
  if len(i.split())>q_len:
    q_len=len(i.split())
print(q_len)

40


In [ ]:
tokenizer=T5TokenizerFast.from_pretrained('t5-base')
model=T5ForConditionalGeneration.from_pretrained('t5-base',return_dict=True)
optimizer=Adam(model.parameters(),lr=0.00001)
q_len=40
t_len=43
batch_size=4
device= 'cuda'if torch.cuda.is_available() else 'cpu'
epochs=5

In [ ]:
class QADataset(Dataset):
  def __init__(self,data,tokenizer,q_len,t_len):
    self.data=data
    self.tokenizer=tokenizer
    self.q_len=q_len
    self.t_len=t_len
    self.question=self.data['question'].tolist()
    self.answer=self.data['answers'].tolist()
    self.context=self.data['context'].tolist()


  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    question=self.question[idx]
    answer=self.answer[idx]
    context=self.context[idx]

    #print(question)
    #print(answer)
    #print(context)

    question_token=self.tokenizer(question,max_length=self.q_len,padding='max_length',truncation=True,pad_to_max_length=True,add_special_tokens=True)
    answer_token=self.tokenizer(answer,max_length=self.t_len,padding='max_length',truncation=True,pad_to_max_length=True,add_special_tokens=True)

    labels=answer_token['input_ids']
    labels[labels==0]=-100

    return {
        "inputs_ids":torch.tensor(question_token['input_ids'],dtype=torch.long),
        "attention_mask":torch.tensor(question_token['attention_mask'],dtype=torch.long),
        "labels":torch.tensor(labels,dtype=torch.long),
        "decoder_attention_mask":torch.tensor(answer_token['attention_mask'],dtype=torch.long)
    }




In [ ]:
sample_data=QADataset(data,tokenizer,q_len,t_len)

In [ ]:
sample_data[0]

{'inputs_ids': tensor([  304,  4068,   410,     8, 16823,  3790,     3, 18280,  2385,    16,
           507,  3449,    16,   301,  1211,  1395,  1410,    58,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([-100, 8942,    9,   26, 1954,  264, 8371, 8283,    1,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0]),
 'decoder_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
train_data,val_data=train_test_split(data,test_size=0.2,random_state=42)

train_dataset=QADataset(train_data,tokenizer,q_len,t_len)
val_dataset=QADataset(val_data,tokenizer,q_len,t_len)

train_data_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_data_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
next(iter(train_data_loader))

{'inputs_ids': tensor([[  571,    19, 11901,    31,     7,  1424,   512,     3,  8232,    58,
              1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [ 2645,   410,    37,  3068, 15884,  1827, 22324,    13,    16,  6622,
             58,     1,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  363,    19,     8,  2015,   496,   358,    16,  1117,  5089,    58,
              1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [ 1029,  4068,

In [ ]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
train_loss=0
val_loss=0
train_batch_count=0
val_batch_count=0

for epoch in range(epochs):
  model.train()
  for batch in tqdm(train_data_loader):
    inputs_ids=batch['inputs_ids'].to(device)
    attention_mask=batch['attention_mask'].to(device)
    labels=batch['labels'].to(device)
    decoder_attention_mask=batch['decoder_attention_mask'].to(device)


    output=model(input_ids=inputs_ids,
                 attention_mask=attention_mask,
                 labels=labels,
                 decoder_attention_mask=decoder_attention_mask)
    #print(output)
    optimizer.zero_grad()
    loss=output.loss
    loss.backward()
    optimizer.step()
    train_loss+=loss.item()
    train_batch_count+=1
    #print(loss)

  print(f"Train Loss: {train_loss/train_batch_count}")

In [ ]:
model.eval()
for batch in tqdm(val_data_loader):
  inputs_ids=batch['inputs_ids'].to(device)
  attention_mask=batch['attention_mask'].to(device)
  labels=batch['labels'].to(device)
  decoder_attention_mask=batch['decoder_attention_mask'].to(device)
  with torch.no_grad():
    output=model(input_ids=inputs_ids,
                 attention_mask=attention_mask,
                 labels=labels,
                 decoder_attention_mask=decoder_attention_mask)
    optimizer.zero_grad()
    loss=output.loss
    val_loss+=loss.item()
    val_batch_count+=1
  print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

  0%|          | 4/4380 [00:00<04:10, 17.49it/s]

1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.3200398683547974
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.3199824690818787
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.2340755065282185
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.3304593563079834


  0%|          | 8/4380 [00:00<03:57, 18.43it/s]

1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.6487142086029052
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.5951141516367595
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.4948115178516932
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.500336304306984


  0%|          | 12/4380 [00:00<04:01, 18.10it/s]

...

  2%|▏         | 90/4380 [00:05<06:27, 11.07it/s]

1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.5534977588388654
1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.5485746814654424


  2%|▏         | 92/4380 [00:05<04:36, 15.50it/s]

1/2 -> Train loss: 6.0496930536234155	Validation loss: 1.5575156270161918


KeyboardInterrupt: 

In [ ]:
model.save_pretrained('qa_model')
tokenizer.save_pretrained('qa_model')


('qa_model/tokenizer_config.json',
 'qa_model/special_tokens_map.json',
 'qa_model/spiece.model',
 'qa_model/added_tokens.json',
 'qa_model/tokenizer.json')

In [ ]:
def predict_ans(context,question,ref_ans=None):
  inputs=tokenizer(question,context,max_length=q_len,padding='max_length',truncation=True,add_special_tokens=True).to(device)
  inputs_ids=torch.tensor(inputs['input_ids'],dtype=torch.long).to(device).unsqueeze(0)
  attention_mask=torch.tensor(inputs['attention_mask'],dtype=torch.long).to(device).unsqueeze(0)
  output=model.generate(input_ids=inputs_ids,attention_mask=attention_mask)
  predicted_ans=tokenizer.decode(output[0],skip_special_tokens=True)
  if ref_ans:
    rouge=evaluate.load('google_bleu')
    #score=bleu.compute(predictions=[predicted_ans],references=[ref_ans])
    print("context: \n",context,"\n")
    print("question: \n",question,"\n")
    return{
      "reference answer: ": ref_ans,
      "predicted answer: ": predicted_ans
      #"bleu score: ": score['bleu']
    }
  else:
    return{
      "context: ": context,
      "question: ": question,
      "predicted answer: ": predicted_ans
    }

In [ ]:
context=data.iloc[0]['context']
question=data.iloc[0]['question']
ref_ans=data.iloc[0]['answers']

In [ ]:
predict_ans(context,question,ref_ans)